In [104]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
import regex as re
from splinter import Browser

In [105]:
url = "https://www.wayfair.com/appliances/sb0/built-in-dishwashers-c1874332.html"

In [208]:
# Set up Splinter
browser = Browser('chrome')

In [162]:
# 
browser.visit(url)

In [108]:
# Parse the HTML
# html = browser.html
# html_soup = BeautifulSoup(html, 'html.parser')

# Start

In [109]:

#results = html_soup.find_all(attrs={"data-hb-id": "Grid.Item", "class":"kzv0b81_6101"})

In [160]:
def scrape_page(results,page):    
    dishwashers = []
    for result in results:
        dw_info = {}
        # Extract the product headline
        title = result.find('h2', class_='kb51y90_6101 kb51y91_6101 kb51y93_6101')
        title = result.find('h2', class_='kb51y90_6101 kb51y91_6101 kb51y93_6101')
        
        if title:
            
            # Grab the maker
            dw_info["maker"] = result.find(class_="oakhm610g_6101 oakhm614l_6101 nhya890_6101").text

            #Grab the size
            size = re.findall(r'[0-9]+"|[0-9]+ in', title.text)
            if size:
                dw_info["size"]=size[0][:2]
            else:
                dw_info["size"]=None

            # Grab noise level; looked it up and all dishwashers have a noiselevel between 30-100 dBA
            noise_level = re.findall(r"[0-9]+ dBA|[0-9]+ Decibel dBA", title.text)
            if noise_level:
                dw_info["noise_level"]=noise_level[0][:2]
                
            else:
                dw_info["noise_level"] = None

            # Grab color options
            color_info = result.find(class_="MoreOptionsText")
            if color_info:
                dw_info["num_colors"] = re.findall(r"\+[0-9]+",color_info.text)
            else:
                dw_info["num_colors"] = 1

            # Grab the elements in the additional information box and store the elements in a list to check through 
            box = result.find_all("p", attrs={"data-hb-id":"Text"})
            box_elements = []
            for x in box:
                box_elements.append(x.text)

            # Grab the number of place settings
            if "Number of Place Settings" in box_elements:
                # Need to grab the index after that of the "Number of Place Settings"
                dw_info["num_place_settings"]= box_elements[box_elements.index("Number of Place Settings") +1]
            else:
                # If there isn't "Number of Place Settings" in the box, then put none
                dw_info["num_place_settings"] = None

            # Grab the number of cycles    
            if "Number of Cycles" in box_elements:
                # Need to grab the index after that of the "Number of Cycles"
                dw_info["num_cycles"]= box_elements[box_elements.index("Number of Cycles") +1]
            else:
                # If there isn't "Number of Cycles" in the box, then put none
                dw_info["num_cycles"] = None

            # Grab the price
            price_info = result.find_all(class_="oakhm627_6101 oakhm6y5_6101 oakhm610g_6101 oakhm6aj_6101")
            prices = []
            for x in price_info:
                y = x.text
                if re.search(r"\$", y):
                    prices.append(y.replace("$",""))
            prices_int = [float(p.replace(",","")) for p in prices]
            if len(prices) ==2:
                dw_info["price"] = prices_int[0]
                dw_info["original_price"] = prices_int[1]
                dw_info["discount"] = prices_int[1]-prices_int[0]
            else:
                dw_info["price"] = prices_int[0]
                dw_info["original_price"] = None
                dw_info["discount"] = 0
            

            # Grab the rating
            rating_info = result.find(class_="_1xxktfu7_6101 _1xxktfu3_6101 _1xxktfu8_6101")
            if rating_info:
                rating_style = re.findall(r"width:[0-9]+|width: [0-9]+", rating_info["style"]) 
                dw_info["rating"]=int(rating_style[0][-2:])/20
            else: 
                dw_info["rating"] =None

            # Grab the number of ratings:
            num_ratings_info = result.find(class_="_1xxktfua_6101 undefined")
            if num_ratings_info:
                dw_info["num_ratings"] = num_ratings_info.text.replace("(", "").replace(")","")
            else:
                dw_info["num_ratings"]=None

            # Add the dishwasher to the list
            dishwashers.append(dw_info)
            
    return dishwashers

  

In [163]:
# Use a for loop to visit the 3 pages
all_pages = []
for x in range(1, 4):
    html = browser.html
    html_soup = BeautifulSoup(html, 'html.parser')
    results = html_soup.find_all(attrs={"data-hb-id": "Grid.Item", "class":"kzv0b81_6101"})
    print(x)
    all_pages.append(scrape_page(results,x))

    if x != 3:
        browser.links.find_by_partial_text('Next').click()

1
2
3


In [209]:
url = "https://www.wayfair.com/appliances/sb0/built-in-dishwashers-c1874332.html?curpage=2"
browser.visit(url)

In [210]:
results = html_soup.find_all(attrs={"data-hb-id": "Grid.Item", "class":"kzv0b81_6101"})

In [205]:
results = html_soup.find_all("a", class_="_1yxeg5wb_6101 _1yxeg5wd_6101")

In [213]:
for result in results:
    title = result.find('h2', class_='kb51y90_6101 kb51y91_6101 kb51y93_6101')
    print(title)

    

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


# Make a dataframe

In [166]:
all_washers = pd.concat([pd.DataFrame(all_pages[0]),pd.DataFrame(all_pages[1]),pd.DataFrame(all_pages[2])], axis=0)
all_washers

,maker,size,noise_level,num_colors,num_place_settings,num_cycles,price,original_price,discount,rating,num_ratings
0,GE Appliances,24,55,[+3],14,4,404.98,599.00,194.02,4.0,806
1,GE Appliances,24,48,[+5],16,5,552.60,849.00,296.40,4.5,8161
2,Samsung,24,39,[+2],15,7,829.00,1199.00,370.00,4.5,1384
3,Samsung,24,55,1,14,4,513.63,649.00,135.37,4.5,617
4,GE Appliances,24,46,[+1],16,5,899.00,999.00,100.00,4.5,3612
5,Samsung,24,48,[+2],15,6,587.96,899.00,311.04,4.0,520
6,GE Appliances,24,59,[+2],12,4,494.00,549.00,55.00,3.5,2215
7,EdgeStar,18,52,[+2],8,6,499.00,NaN,0.00,4.5,162
8,Samsung,24,48,[+1],15,6,587.96,899.00,311.04,4.0,813
9,GE Profile™,24,39,[+1],16,6,1484.00,NaN,0.00,4.5,955


# Start parsing

In [207]:
browser.quit()